# CMB-HD Forecasting Data

This notebook serves as an example of how to load in the CMB-HD data contained in this repository.

Latest version: `v1.1`

In [1]:
from hd_mock_data import hd_data

## Using the most recent data

The CMB-HD data is accessed using the `hdMockData` class in `hd_data.py`. The `hdMockData` class takes in a single variable, the `version`, which is set to `version = 'latest'` by default. This is what we will use below:

In [2]:
datalib = hd_data.HDMockData()

The minimum and maximum multipoles are stored as attributes of the class:

In [3]:
print(f'lmin for TT, TE, EE, BB: {datalib.lmin}')
print(f'lmax for TT: {datalib.lmaxTT}')
print(f'lmax for TE, EE, BB: {datalib.lmax}')
print(f'(Lmin, Lmax) for lensing: ({datalib.Lmin}, {datalib.Lmax})')

lmin for TT, TE, EE, BB: 30
lmax for TT: 40000
lmax for TE, EE, BB: 20100
(Lmin, Lmax) for lensing: (30, 20100)


## Binning

### Load the binning file

The `bin_edges` method returns a 1D array holding the bin edges:

In [4]:
bin_edges = datalib.bin_edges()
print(bin_edges.shape)

(171,)


### Getting a binning matrix

The `binning_matrix` method takes in two optional integer arguments, `lmin` and `lmax`. By default, these are set to the `lmin` and `lmaxTT` values printed out above.

In [5]:
binning_matrix = datalib.binning_matrix()
print(binning_matrix.shape)

(103, 20099)


In [6]:
binning_matrix_lmax5000 = datalib.binning_matrix(lmax=5000)
print(binning_matrix_lmax5000.shape)

(52, 4999)


## CMB-HD lensed or delensed $TT$, $TE$, $EE$, $BB$, and lensing $\kappa\kappa$ power spectra

The `cmb_theory_spectra` method takes in three arguments:
1. `cmb_type`: either `'lensed'` or `'delensed'`
2. `baryonic_feedback` (optional, default=`False`): `True` to include the effects of baryonic feedback, or `False` otherwise
3. `output_lmax` (optional, default=`None`): passing an integer value to `output_lmax` will return the spectra only up to that multipole, otherwise all multipoles are returned.

It returns a dictionary with the following keys and values:
- `'ells'`: holds a 1D array of the multipoles for the spectra
- `'tt'`, `'te'`, `'ee'`, `'bb'`: 1D arrays of lensed or delensed CMB power spectra
- `'kk'`: 1D array holding the CMB lensing power spectrum

The spectra are unbinned, starting at $\ell = 0$. The CMB spectra are in $C_\ell$'s, i.e. without any $\ell(\ell+1)/2\pi$ factors applied, in units of $\mu$K$^2$. The CMB lensing power spectrum uses the convention $C_L^{\kappa\kappa} = [L(L+1)]^2 C_L^{\phi\phi} / 4$. 
- These conventions apply to all spectra (e.g. noise, foregrounds) that are returned by methods of the `HDMockData` class.


In [7]:
delensed_theo = datalib.cmb_theory_spectra('delensed')
print(delensed_theo.keys())
print(delensed_theo['tt'].shape)
print(delensed_theo['ells'])

dict_keys(['ells', 'tt', 'te', 'ee', 'bb', 'kk'])
(40001,)
[0.0000e+00 1.0000e+00 2.0000e+00 ... 3.9998e+04 3.9999e+04 4.0000e+04]


In [8]:
delensed_theo_lmax5000 = datalib.cmb_theory_spectra('delensed', output_lmax=5000)
print(delensed_theo_lmax5000['tt'].shape)
print(delensed_theo_lmax5000['ells'])

(5001,)
[0.000e+00 1.000e+00 2.000e+00 ... 4.998e+03 4.999e+03 5.000e+03]


## Residual extragalactic foreground power spectra

### Individual foreground components at each frequency

We have residual extragalactic foreground spectra at 90 and 150 GHz for the tSZ and kSZ effects, the CIB, and radio sources.

The `fg_spectra` method takes in two arguments:
1. `freq`: the frequency, either `'f090'` for 90 GHz or `'f150'` for 150 GHz.
2. `output_lmax` (optional, default=`None`) : passing an integer value to `output_lmax` will return the spectra only up to that multipole, otherwise all multipoles are returned.

It returns a dictionary with a key `'ells'` holding a 1D array of the multipoles, and keys `'tsz'`, `'ksz'`, `'cib'`, and `'radio'`. The spectra are unbinned starting at $\ell = 0$ and are in $C_\ell$'s, i.e. without any $\ell(\ell+1)/2\pi$ factors applied, with units of $\mu$K$^2$.
- Note: in version `v1.0`, the kSZ only included the reionization component. In later versions, it includes both reionization and late-time contributions.

In [9]:
fgs_f090 = datalib.fg_spectra('f090')
print(fgs_f090.keys())
print(fgs_f090['ells'])

dict_keys(['ells', 'ksz', 'tsz', 'cib', 'radio'])
[0.e+00 1.e+00 2.e+00 ... 3.e+04 3.e+04 3.e+04]


#### Loading in the kSZ power spectrum (only `v1.1` and later) 

In `v1.1`, there is an option to load the total (frequency-independent) kSZ power spectrum apart from the other frequency-dependent foregrounds. The method `cl_ksz` takes a single optional argument, the `output_lmax` (same as above for the individual components), and returns two 1D arrays holding the multipoles and the kSZ power spectrum in units of $\mu$K^2.

In [10]:
ksz_ells, cl_ksz = datalib.cl_ksz()
print(cl_ksz.shape)

(40001,)


### Coadded total residual foreround power spectrum

The `coadded_fg_spectrum` method returns two 1D arrays: the multipoles, and the sum of the foreground components coadded over the two frequencies (in units of $\mu$K$^2$). It takes a single optional argument, the `output_lmax` (same as above for the individual components).

In [11]:
fg_ells, cl_fg = datalib.coadded_fg_spectrum()
print(cl_fg.shape)

(40001,)


## Noise power spectra

### Instrumental noise for $TT$, $TE$, $EE$, $BB$ at each frequency

The method `white_noise_cls` returns a dictionary holding the beam-deconvolved instrumental noise power spectra at either 90 or 150 GHz. It takes in two arguments:
1. `freq`: the frequency, either `'f090'` for 90 GHz or `'f150'` for 150 GHz.
2. `output_lmax` (optional, default=`None`) : passing an integer value to `output_lmax` will return the spectra only up to that multipole, otherwise all multipoles are returned.

The returned dictionary has keys `'ells'`, `'tt'`, `'te'`, `'ee'`, `'bb'`. The noise spectra are in units of $\mu$K$^2$ in $C_\ell$'s. 

In [12]:
white_noise_f090 = datalib.white_noise_cls('f090')
print(white_noise_f090.keys())

dict_keys(['tt', 'te', 'ee', 'bb', 'ells'])


### Coadded noise for $TT$, $TE$, $EE$, $BB$

The method `cmb_noise_spectra` returns a dictionary holding the beam-deconvolved instrumental noise power spectra, coadded from 90 and 150 GHz. It takes in two optional arguments:
1. `include_fg` (optional, default=`True`): Whether to include residual extragalactic foregrounds in the $TT$ noise power spectrum.
2. `output_lmax` (optional, default=`None`) : passing an integer value to `output_lmax` will return the spectra only up to that multipole, otherwise all multipoles are returned.

The returned dictionary has keys `'ells'`, `'tt'`, `'te'`, `'ee'`, `'bb'`. The noise spectra are in units of $\mu$K$^2$ in $C_\ell$'s. 

In [13]:
noise_cls = datalib.cmb_noise_spectra()
print(noise_cls.keys())
print(noise_cls['tt'].shape)

dict_keys(['ells', 'tt', 'te', 'ee', 'bb'])
(40001,)


### Lensing noise

The `lensing_noise_spectrum` spectrum returns two 1D arrays, `L` holding the multipoles and `nlkk` holding the noise on $C_L^{\kappa\kappa} = [L(L+1)]^2 C_L^{\phi\phi} / 4$.

It takes in a single optional argument, `output_Lmax` (note the capital "L").

Note that, as of `v1.1`, the lensing noise is only stored up to $L_\mathrm{max} = 20,100$.

In [14]:
L, nlkk = datalib.lensing_noise_spectrum()
print(nlkk.shape)

(20101,)


## Covariance matrices

### 5 x 5 block covariance matrices for $TT$, $TE$, $EE$, $BB$, $\kappa\kappa$ over the range $\ell \in [30, 20000]$

The CMB-HD covariance matrices have shape $5n_\mathrm{bin} \times 5n_\mathrm{bin}$, where $n_\mathrm{bin}$ is the number of bins between $\ell_\mathrm{min} = 30$ and $\ell_\mathrm{max} = 20,100$. The full covariance matrix $\mathbb{C}$ takes the form $$\mathbb{C} = \begin{pmatrix} \mathbb{C}^{TT,TT} & \mathbb{C}^{TT,TE} & \cdots & \mathbb{C}^{TT,\kappa\kappa} \\ \mathbb{C}^{TE, TT} & \mathbb{C}^{TE, TE} & \cdots & \mathbb{C}^{TE,\kappa\kappa} \\ \vdots & \vdots & \ddots & \vdots \\ \mathbb{C}^{\kappa\kappa,TT} & \mathbb{C}^{\kappa\kappa,TE} & \cdots & \mathbb{C}^{\kappa\kappa,\kappa\kappa} \end{pmatrix},$$ where each block $\mathbb{C}^{XY,WZ}$ has shape $n_\mathrm{bin} \times n_\mathrm{bin}$, and $XY$ and $WZ$ are each one of $\{TT,TE,EE,BB,\kappa\kappa\}$. 
- The elements of the individual blocks are $\mathbb{C}_{\ell_b \ell_b'}^{XY,WZ} = \mathrm{cov}\left(C_{\ell_b}^{XY}, C_{\ell_b'}^{WZ}\right)$, where $\ell_b$ and $\ell_b'$ are multipole bin centers.
- The order of blocks along the rows/columns is $TT$, $TE$, $EE$, $BB$, $\kappa\kappa$.
- The CMB x CMB blocks have units of $\mu$K$^4$, the CMB x lensing blocks have units of $\mu$K$^2$, and the lensing x lensing block is dimensionless.


The `block_covmat` method returns the binned block covariance matrix described above as a 2D array. It takes in a single argument, `cmb_type`, which should be either `'lensed'` or `'delensed'`. 

In [15]:
delensed_cov = datalib.block_covmat('delensed')
print(delensed_cov.shape)

(515, 515)


### Diagonal $TT \times TT$ covariance matrix over the range $\ell \in (20000, 40000]$ (only `v1.1` and later) 

When extending $\ell_\mathrm{max}^{TT}$ to 40,000, we approximate the covariance matrix as diagonal in the range $\ell \in (20000, 40000]$, with $\mathbb{C}_{\ell_b \ell_b'}^{TT,TT} = \delta_{\ell_b \ell_b'} \mathrm{var}(C_{\ell_b}^{TT})$, using the same notation as above.

The method `tt_diag_covmat` returns the binned $TT \times TT$ covariance matrix in the range $\ell \in (20000, 40000]$. It takes in a single argument, `cmb_type`, which should be either `'lensed'` or `'delensed'`. 

In [16]:
delensed_tt_cov = datalib.tt_diag_covmat('delensed')
print(delensed_tt_cov.shape)

(66, 66)


---

## Fixing the data version

You may access older versions of the data by passing the correct `version`. 
- __Note__: this should only be done to reproduce published results that were obtained using a previous version of the data.

For example, to access the data used in [MacInnis, Sehgal, and Rothermel (2023)](https://arxiv.org/abs/2309.03021), use `version='v1.0'`:

In [17]:
datalib_fixed_version = hd_data.HDMockData(version='v1.0')
print(f'For version {datalib_fixed_version.version}:')
print(f'lmin for TT, TE, EE, BB: {datalib_fixed_version.lmin}')
print(f'lmax for TT: {datalib_fixed_version.lmaxTT}')
print(f'lmax for TE, EE, BB: {datalib_fixed_version.lmax}')
print(f'(Lmin, Lmax) for lensing: ({datalib_fixed_version.Lmin}, {datalib_fixed_version.Lmax})')

For version v1.0:
lmin for TT, TE, EE, BB: 30
lmax for TT: 20100
lmax for TE, EE, BB: 20100
(Lmin, Lmax) for lensing: (30, 20100)
